In [153]:
import ghmm
from collections import OrderedDict
import cPickle as pickle
import numpy as np
from itertools import product as iterproduct, chain
from pprint import pprint
import pysam
import os
import pandas
from copy import deepcopy
import re
import editdistance
import sys
import math
import random
from nbwrapper import getargs
from multiprocessing import Pool
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/lib/alignment_validation.ipynb"
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/lib/alignment_lib.ipynb"
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/lib/analysis_lib.ipynb"

In [154]:
args = getargs()

In [155]:
NMERS = 6

In [156]:
args = {
    "events" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/loman006-1_100.events.template.pickle",
    "out_basename" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/loman006-1_100.called",
    "ref": "/home/ibis/gregor.sturm/nanopore/NanoporeData/PublicData/LomanLab_MAP-006/ecoli_mg1655.fa",
    "hmm_params": "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/loman006-1.model.computed.pickle",
    "ncores": 62,
    "nmers": NMERS,
    "multivariate": False
}

In [157]:
NMERS = int(args["nmers"])
NSTATES = 4**NMERS
MULTIVARIATE = bool(int(args["multivariate"]))
args["ncores"] = int(args["ncores"])

In [158]:
HMM_PARAMS = pickle.load(open(args["hmm_params"], 'rb'))
HMM_PARAMS = HMM_PARAMS["/opt/chimaera/model/r7.3_e6_70bps_6mer/template_median68pA.model"]
ALL_KMERS = ["".join(x) for x in iterproduct("ACGT", repeat=NMERS)]
assert HMM_PARAMS["kmer"].tolist() == ALL_KMERS

# Train Model 

In [159]:
def mk_transmat1(nmers):
    """make a transition matrix assuming move=1"""
    n_components = len(ALL_KMERS)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(ALL_KMERS):
        for i, to_kmer in enumerate(ALL_KMERS):
            p = 1/4. if from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)] else 0.
            transmat[j, i] = p          
            
    return transmat.tolist()

In [160]:
def mk_transmat0(nmers):
    """make a transition matrix assuming move=0 or move=1"""
    n_components = len(ALL_KMERS)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(ALL_KMERS):
        for i, to_kmer in enumerate(ALL_KMERS):
            p = 0
            if from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)]:
                """move=1"""
                p = (9/10.) * (1/4.) 
            elif from_kmer == to_kmer:
                """move=0"""
                p = (1/10.) * 1
            transmat[j, i] = p          
            
    return transmat.tolist()

In [161]:
def mk_transmat2(nmers):
    """make a transition matrix assuming move=0 or move=1 or move=2"""
    n_components = len(ALL_KMERS)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(ALL_KMERS):
        for i, to_kmer in enumerate(ALL_KMERS):
            p = 0
            if from_kmer[-(NMERS-2):] == to_kmer[:(NMERS-2)]:
                """move=2"""
                p = (2/50.) * (1/16.)
            elif from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)]:
                """move=1"""
                p = (47/50.) * (1/4.) 
            elif from_kmer == to_kmer:
                """move=0"""
                p = (1/50.) * 1
            transmat[j, i] = p          
            
    return transmat.tolist()

In [162]:
mk_transmat = mk_transmat2

In [163]:
def mk_model_simple(): 
    """ simple model, only taking the means into account. """
    A = mk_transmat(NMERS)
    B = HMM_PARAMS[["level_mean", "level_stdv"]].values.tolist() #mu, std of each state
    pi = [1/float(NSTATES)] * NSTATES   # initial probabilities per state
    # generate model from parameters
    model = ghmm.HMMFromMatrices(F,ghmm.GaussianDistribution(F), A, B, pi)
    return model

In [164]:
F = ghmm.Float()  # emission domain of this model
def mk_model():
    if MULTIVARIATE: 
        return mk_model_multivariate()
    else: 
        return mk_model_simple()

In [165]:
model = mk_model()
s = str(model)
print(s)

GaussianEmissionHMM(N=4096)
  state 0 (initial=0.00, mu=62.96, sigma=0.85)
    Transitions: ->0 (0.00), ->1 (0.00), ->2 (0.00), ->3 (0.00), ->4 (0.00), ->5 (0.00), ->6 (0.00), ->7 (0.00), ->8 (0.00), ->9 (0.00), ->10 (0.00), ->11 (0.00), ->12 (0.00), ->13 (0.00), ->14 (0.00), ->15 (0.00)
  state 1 (initial=0.00, mu=58.16, sigma=0.68)
    Transitions: ->1 (0.02), ->4 (0.23), ->5 (0.23), ->6 (0.23), ->7 (0.23), ->16 (0.00), ->17 (0.00), ->18 (0.00), ->19 (0.00), ->20 (0.00), ->21 (0.00), ->22 (0.00), ->23 (0.00), ->24 (0.00), ->25 (0.00), ->26 (0.00), ->27 (0.00), ->28 (0.00), ->29 (0.00), ->30 (0.00), ->31 (0.00)

  ...

  state 4094 (initial=0.00, mu=46.66, sigma=0.83)
    Transitions: ->4064 (0.00), ->4065 (0.00), ->4066 (0.00), ->4067 (0.00), ->4068 (0.00), ->4069 (0.00), ->4070 (0.00), ->4071 (0.00), ->4072 (0.00), ->4073 (0.00), ->4074 (0.00), ->4075 (0.00), ->4076 (0.00), ->4077 (0.00), ->4078 (0.00), ->4079 (0.00), ->4088 (0.23), ->4089 (0.23), ->4090 (0.23), ->4091 (0.23), ->409

In [166]:
def result_to_seq(result):
    states = result[0]
    kmers = [ALL_KMERS[x] for x in states]
    seq = [kmer[0] for kmer in kmers] + [kmers[-1][1:]]
    return "".join(seq)

In [167]:
def predict(mixed):
    """mixed is a set of tuples (event_mean, event_stdv)"""
    if MULTIVARIATE: 
        emissions = [x for x in chain(*mixed)]    
    else: 
        emissions = [x[0] for x in mixed]
    seq = ghmm.EmissionSequence(F, emissions)
    result = model.viterbi(seq)
    return result_to_seq(result)

In [168]:
s = model.sampleSingle(10)
s = [x for x in s]
seq = zip([s[i] for i in range(0, len(s), 2)], [s[i] for i in range(1, len(s), 2)])

In [169]:
predict(seq)

'TCGATGGATC'

In [170]:
model

# Validate Model 

In [171]:
!pwd

/home/ibis/gregor.sturm/nanopore/own/notebooks


In [172]:
assert os.path.isfile(args["events"])

In [173]:
ref = load_ref(args["ref"])

['>gi|556503834|ref|NC_000913.3| Escherichia coli str. K-12 substr. MG1655, complete genome']
AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAACTGGTTACCTGCCGTGAGTAAAT


In [174]:
file_data = pickle.load(open(args["events"], 'rb'))
file_data = [f for f in file_data if f is not None]

In [175]:
prepare_filemap(file_data)

In [176]:
def train_read(file_obj):    
    events = [x["mean"] for x in file_obj["events"].to_dict("records")]
    seq = ghmm.EmissionSequence(F, events)
    model.baumWelch(seq)
    

In [177]:
def basecall_read(file_obj):
    file_obj = normalize_read(file_obj, target_median=59)
    events = [(x["mean"], x["stdv"]) for x in file_obj["events"].to_dict("records")]
    called_seq = predict(events)
    return (file_obj["channel"], file_obj["file_id"], called_seq)

In [178]:
# """ train with baum-welch """
# for i, file_obj in enumerate(file_data): 
#     sys.stdout.write('\rdone {0:%}'.format(i/float(len(file_data))))
#     train_read(file_obj)

In [179]:
p = Pool(args["ncores"])

In [180]:
print("Prediction: ")
results = []
try:
    for i, res in enumerate(p.imap_unordered(basecall_read, file_data), 1):
        results.append(res)
        sys.stdout.write('\rdone {0:%}'.format(i/float(len(file_data))))
    p.close()
    p.join()
except KeyboardInterrupt:
    p.terminate()

Prediction: 
done 100.000000%

### Stats

In [181]:
types = ["metrichor", "called", "random"]
fasta_files = {t: "{0}.{1}.fa".format(args["out_basename"], t) for t in types}

In [182]:
## metrichor fasta
with open(fasta_files["metrichor"], 'w') as f: 
    for file_obj in file_data: 
        f.write(">ch{0}_file{1}_metrichor".format(file_obj["channel"], file_obj["file_id"])+ "\n")
        f.write(file_obj["fastq"].split("\n")[1] + "\n")

In [183]:
## called fasta/random fasta
with open(fasta_files["called"], 'w') as f: 
    with open(fasta_files["random"], 'w') as fr:
        for channel, file_id, seq in results: 
            f.write(">ch{0}_file{1}_called".format(channel, file_id)+ "\n")
            fr.write(">ch{0}_file{1}_random".format(channel, file_id)+ "\n")
            f.write(seq + "\n")
            fr.write("".join([random.choice("ACGT") for _ in range(len(seq))]))

In [184]:
for t in types: 
    sam_file = "{0}.{1}.sam".format(args["out_basename"], t)
    graphmap(args["ref"], fasta_files[t], sam_file, args["ncores"])
    prepare_sam("{0}.{1}".format(args["out_basename"], t))

[Index 17:59:28] Running in fast and sensitive mode. Two indexes will be used (double memory consumption).
[Index 17:59:28] Index already exists. Loading from file.
[Index 17:59:29] Secondary index already exists. Loading from file.
[Index 17:59:30] Index loaded in 1.02 sec.
[Index 17:59:30] Memory consumption: [currentRSS = 674 MB, peakRSS = 1113 MB]

[Run 17:59:30] Automatically setting the maximum allowed number of regions: max. 500, attempt to reduce after 100
[Run 17:59:30] Reference genome is assumed to be linear.
[Run 17:59:30] Only one alignment will be reported per mapped read.
[ProcessReads 17:59:30] Reads will be loaded in batches of up to 200 MB in size.
[ProcessReads 17:59:30] Batch of 100 reads (0 MiB) loaded in 0.01 sec. (16786664 bases)
[ProcessReads 17:59:30] Memory consumption: [currentRSS = 675 MB, peakRSS = 1113 MB]
[ProcessReads 17:59:30] Using 62 threads.
[ProcessReads 17:59:33] [CPU time: 40.88 sec, RSS: 698 MB] Read: 100/100 (100.00%) [m: 100, u: 0]             

In [185]:
def mk_stat(t):
    samfile = "{0}.{1}.sorted.bam".format(args["out_basename"], t)
    sst = samstats(samfile, ref, ncores=args["ncores"])
    return pandas.DataFrame(sst.print_summary())

In [186]:
stats = map(mk_stat, types)
print(types)
side_by_side(*stats)

['metrichor', 'called', 'random']


,0,1,2,3
0,mapped_reads/total_reads,100,100,100.000000%
1,significant_reads/total_reads,99,100,99.000000%
2,mapped_nts/total_nts,865397,970301,89.188510%
3,editdistance/alignment_length,311257,948375,32.820034%
4,alignment_score/alignment_length,1802584,948375,190.070805%
5,SNPs/mapped_nts,123375,865397,14.256463%
6,ins/mapped_nts,104806,865397,12.110742%
7,del/mapped_nts,82978,865397,9.588432%
,0,1,2,3
0,mapped_reads/total_reads,99,100,99.000000%


In [187]:
# for t, df in zip(types, stats):
#     with open("{0}.stats.{1}.html".format(args["out_basename"], t), 'w') as f:
#         f.write(df.to_html())

In [188]:
# def score_consensus(t):
#     consensus = mk_consensus("{0}.{1}.sorted.bam".format(args["out_basename"], t), ref_file)
#     return(consensus)
#     consensus = consensus.split("\n")[1].to_upper()
#     score = needle(ref, consensus)
#     return (consensus, score)

In [189]:
# p = Pool(args["ncores"])
# try:
#     consensus = p.map(score_consensus, types)
#     p.close()
# except KeyboardInterrupt:
#     p.terminate()

In [190]:
# consensus

In [191]:
# mk_consensus("{0}.{1}.sorted.bam".format(args["out_basename"], "metrichor"), ref_file)